# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 12:43PM,242549,10,8308949,"ClearSpec, LLC",Released
1,Jul 22 2022 12:43PM,242549,10,8308952,"ClearSpec, LLC",Released
2,Jul 22 2022 12:43PM,242549,10,8308954,"ClearSpec, LLC",Released
3,Jul 22 2022 12:39PM,242548,18,AYRTON-8307858,"Ayrton Saunders, Inc.",Released
4,Jul 22 2022 12:38PM,242547,18,AYRTON-8307856,"Ayrton Saunders, Inc.",Released
5,Jul 22 2022 12:28PM,242546,102,PBR-072022-013,"Senseonics, Incorporated",Released
6,Jul 22 2022 12:25PM,242545,10,MSP212445,"Methapharm, Inc.",Released
7,Jul 22 2022 12:25PM,242545,10,MSP212444,"Methapharm, Inc.",Released
8,Jul 22 2022 12:25PM,242545,10,MSP212453,"Methapharm, Inc.",Released
9,Jul 22 2022 12:25PM,242545,10,MSP212454,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,242545,Released,6
18,242546,Released,1
19,242547,Released,1
20,242548,Released,1
21,242549,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242545,NaN,6.0
242546,NaN,1.0
242547,NaN,1.0
242548,NaN,1.0
242549,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242450,0.0,4.0
242464,0.0,1.0
242467,0.0,1.0
242473,0.0,41.0
242494,2.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242450,0,4
242464,0,1
242467,0,1
242473,0,41
242494,2,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242450,0,4
1,242464,0,1
2,242467,0,1
3,242473,0,41
4,242494,2,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242450,,4
1,242464,,1
2,242467,,1
3,242473,,41
4,242494,2,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC"
3,Jul 22 2022 12:39PM,242548,18,"Ayrton Saunders, Inc."
4,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc."
5,Jul 22 2022 12:28PM,242546,102,"Senseonics, Incorporated"
6,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc."
12,Jul 22 2022 12:00PM,242540,10,Eywa Pharma Inc.
25,Jul 22 2022 11:47AM,242539,21,"NBTY Global, Inc."
26,Jul 22 2022 11:38AM,242537,12,"SD Head USA, LLC"
27,Jul 22 2022 10:34AM,242536,10,"ACI Healthcare USA, Inc."
31,Jul 22 2022 10:27AM,242534,20,Alumier Labs Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",,3
1,Jul 22 2022 12:39PM,242548,18,"Ayrton Saunders, Inc.",,1
2,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",,1
3,Jul 22 2022 12:28PM,242546,102,"Senseonics, Incorporated",,1
4,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",,6
5,Jul 22 2022 12:00PM,242540,10,Eywa Pharma Inc.,,13
6,Jul 22 2022 11:47AM,242539,21,"NBTY Global, Inc.",,1
7,Jul 22 2022 11:38AM,242537,12,"SD Head USA, LLC",1,
8,Jul 22 2022 10:34AM,242536,10,"ACI Healthcare USA, Inc.",,4
9,Jul 22 2022 10:27AM,242534,20,Alumier Labs Inc.,3,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",3,
1,Jul 22 2022 12:39PM,242548,18,"Ayrton Saunders, Inc.",1,
2,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",1,
3,Jul 22 2022 12:28PM,242546,102,"Senseonics, Incorporated",1,
4,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",6,
5,Jul 22 2022 12:00PM,242540,10,Eywa Pharma Inc.,13,
6,Jul 22 2022 11:47AM,242539,21,"NBTY Global, Inc.",1,
7,Jul 22 2022 11:38AM,242537,12,"SD Head USA, LLC",,1
8,Jul 22 2022 10:34AM,242536,10,"ACI Healthcare USA, Inc.",4,
9,Jul 22 2022 10:27AM,242534,20,Alumier Labs Inc.,,3


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",3,
1,Jul 22 2022 12:39PM,242548,18,"Ayrton Saunders, Inc.",1,
2,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",1,
3,Jul 22 2022 12:28PM,242546,102,"Senseonics, Incorporated",1,
4,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",6,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",3.0,NaN
1,Jul 22 2022 12:39PM,242548,18,"Ayrton Saunders, Inc.",1.0,NaN
2,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",1.0,NaN
3,Jul 22 2022 12:28PM,242546,102,"Senseonics, Incorporated",1.0,NaN
4,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",6.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",3.0,0.0
1,Jul 22 2022 12:39PM,242548,18,"Ayrton Saunders, Inc.",1.0,0.0
2,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",1.0,0.0
3,Jul 22 2022 12:28PM,242546,102,"Senseonics, Incorporated",1.0,0.0
4,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",6.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1940145,84.0,0.0
102,242546,1.0,0.0
12,242537,0.0,1.0
16,485012,1.0,1.0
18,485095,2.0,0.0
19,242494,1.0,2.0
20,970048,17.0,3.0
21,485003,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1940145,84.0,0.0
1,102,242546,1.0,0.0
2,12,242537,0.0,1.0
3,16,485012,1.0,1.0
4,18,485095,2.0,0.0
5,19,242494,1.0,2.0
6,20,970048,17.0,3.0
7,21,485003,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,84.0,0.0
1,102,1.0,0.0
2,12,0.0,1.0
3,16,1.0,1.0
4,18,2.0,0.0
5,19,1.0,2.0
6,20,17.0,3.0
7,21,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,102,Released,1.0
2,12,Released,0.0
3,16,Released,1.0
4,18,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,18,19,20,21
Status,,,,,,,,
Executing,0.0,0.0,1.0,1.0,0.0,2.0,3.0,0.0
Released,84.0,1.0,0.0,1.0,2.0,1.0,17.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,18,19,20,21
0,Executing,0.0,0.0,1.0,1.0,0.0,2.0,3.0,0.0
1,Released,84.0,1.0,0.0,1.0,2.0,1.0,17.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,18,19,20,21
0,Executing,0.0,0.0,1.0,1.0,0.0,2.0,3.0,0.0
1,Released,84.0,1.0,0.0,1.0,2.0,1.0,17.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()